# Acoustic Wave Equation in an isotropic medium with uniform density

$$\frac{1}{c^2} \frac{\partial^2 p}{\partial t^2} + \nabla . \nabla p = s$$

In [26]:
import numpy as np
%matplotlib inline

from devito import Grid, Function, TimeFunction, SparseTimeFunction, solve, Eq, Operator
from examples.seismic import Model, plot_velocity, TimeAxis, RickerSource, Receiver

In [29]:
dim = 3
shape = (10, 10, 10)
extent = (10, 10, 10)
origin = (0, 0, 0)
grid = Grid(shape=shape, extent=extent, origin=origin)

so = 2 # Space order
to = 2 # Time order

# Create an initial velocity profile
v_initial = np.zeros(shape)
v_initial[:, :, 0] = 1.0

# Set up the model based on our grid and initial velocity
model = Model(origin, extent, shape, so, v_initial)
dt = model.critical_dt

Operator `initdamp` ran in 0.01 s
Operator `pad_vp` ran in 0.01 s


In [31]:
# Set up time-scale
t0 = 0
tn = 1000
time_range = TimeAxis(start=t0, stop=tn, step=dt)

# Now set up a source and receivers
# We will set up a source at the origin (0, 0, 0), and receivers along the line parametrized by (t, 1, 1)n
n_src = 1
n_recv = 5
src_coords = np.zeros((n_src, dim))
recv_coords = np.zeros((n_recv, dim))
for i in range(n_recv):
    recv_coords[i] = (i / (n_recv - 1), 1, 1)

    
f0 = 0.010  # Source peak frequency is 10Hz (0.010 kHz)
src = RickerSource(name='src', grid=model.grid, f0=f0,
                   npoint=1, time_range=time_range)

# First, position source centrally in all dimensions, then set depth
src.coordinates.data[:, :] = src_coords

rec = Receiver(name='rec', grid=model.grid, npoint=n_recv, time_range=time_range)
rec.coordinates.data[:, :] = recv_coords

m = Function(name='m', grid=model.grid, space_order=so)
u = TimeFunction(name='u', grid=model.grid, space_order=so, time_order=to)

pde = model.m * u.dt2 - u.laplace

pde

-Derivative(u(t, x, y, z), (x, 2)) - Derivative(u(t, x, y, z), (y, 2)) - Derivative(u(t, x, y, z), (z, 2)) + Derivative(u(t, x, y, z), (t, 2))/vp(x, y, z)**2